In [9]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from deepface import DeepFace
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

import numpy as np
from deepface import DeepFace

In [10]:
# Check if a GPU is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. PyTorch is using the GPU.")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. PyTorch is using the CPU.")

CUDA is not available. PyTorch is using the CPU.


In [11]:
# Load the YOLOv8 model and move it to the GPU if available
model = YOLO('yolov8n-face.pt').to(device)

# v1

In [ ]:
# Create directory to save detected faces
os.makedirs('just_faces', exist_ok=True)

# Load and prepare the image
image_path = 'test_images_big/DSC_1064 1.JPG'
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for detection

# Perform face detection
results = model(image_rgb)

# Process results and save detected faces
for i, result in enumerate(results):
    boxes = result.boxes

    for j, box in enumerate(boxes):
        xyxy = box.xyxy[0].cpu().numpy()  # Get coordinates as a numpy array
        conf = box.conf[0].cpu().numpy() if box.conf is not None else 0  # Extract confidence

        if conf > 0.1:  # Confidence threshold
            x1, y1, x2, y2 = map(int, xyxy)

            # Extract face
            face_crop = image[y1:y2, x1:x2]

            # Save face crop as an image
            face_path = os.path.join('just_faces', f'face_{i}_{j}.jpg')
            cv2.imwrite(face_path, face_crop)

print("Detected faces have been saved in the 'just_faces' folder.")

# Path to the folder containing detected faces
faces_folder_path = 'just_faces'

# Function to get embeddings for each face image
def get_face_embeddings(face_image_path, model_name='Facenet'):
    # Get the embedding using DeepFace
    embeddings = DeepFace.represent(img_path=face_image_path, model_name=model_name, enforce_detection=False)
    return embeddings

# Get embeddings for all face images in the folder
face_embeddings = []

for face_image_name in os.listdir(faces_folder_path):
    face_image_path = os.path.join(faces_folder_path, face_image_name)
    
    if os.path.isfile(face_image_path):  # Check if it's a file
        try:
            embeddings = get_face_embeddings(face_image_path)
            face_embeddings.append(embeddings[0]['embedding'])
        except Exception as e:
            print(f"Error processing {face_image_path}: {e}")

# Print the embeddings
for i, embedding in enumerate(face_embeddings):
    print(f"Embedding for face {i+1}: {embedding}")

# Optionally, save the embeddings to a file
np.save('face_embeddings.npy', face_embeddings)
print("Face embeddings have been saved to 'face_embeddings.npy'.")


# v2

In [5]:
# Create directories for saving embeddings
os.makedirs('embeddings', exist_ok=True)

# Load and prepare the image
image_path = 'test_images_big/DSC_1064 1.JPG'
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for detection

# Perform face detection
results = model(image_rgb)

# Process results and get embeddings
face_embeddings = []

for result in results:
    boxes = result.boxes

    for box in boxes:
        xyxy = box.xyxy[0].cpu().numpy()  # Get coordinates as a numpy array
        conf = box.conf[0].cpu().numpy() if box.conf is not None else 0  # Extract confidence

        if conf > 0.1:  # Confidence threshold
            x1, y1, x2, y2 = map(int, xyxy)

            # Extract face
            face_crop = image[y1:y2, x1:x2]

            # Convert face crop to RGB and save for embedding extraction
            face_crop_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)

            # Get the embedding using DeepFace
            try:
                embeddings = DeepFace.represent(img_path=face_crop_rgb, model_name='Facenet', enforce_detection=False)
                face_embeddings.append(embeddings[0]['embedding'])
            except Exception as e:
                print(f"Error processing detected face: {e}")

# Save embeddings to a file with the original image name
base_name = os.path.splitext(os.path.basename(image_path))[0]
np.save(os.path.join('embeddings', f'{base_name}.npy'), face_embeddings)
print(f"Face embeddings have been saved to 'embeddings/{base_name}.npy'.")



0: 384x640 6 faces, 49.1ms
Speed: 2.0ms preprocess, 49.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Face embeddings have been saved to 'embeddings/DSC_1064 1.npy'.


# v3

In [12]:
# Create directories for saving embeddings
os.makedirs('embeddings', exist_ok=True)


# Path to the directory containing images
image_dir = 'test_images_big'  # Change this to your directory

# Get all image files in the specified directory
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Loop through all image files with a single progress bar
with tqdm(total=len(image_files), desc="Processing Images", unit="image") as pbar:
    for image_name in image_files:
        image_path = os.path.join(image_dir, image_name)

        # Load and prepare the image
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for detection

        # Perform face detection (set verbose to False if applicable)
        results = model(image_rgb, verbose=False)

        # Process results and get embeddings
        face_embeddings = []

        for result in results:
            boxes = result.boxes

            for box in boxes:
                xyxy = box.xyxy[0].cpu().numpy()  # Get coordinates as a numpy array
                conf = box.conf[0].cpu().numpy() if box.conf is not None else 0  # Extract confidence

                if conf > 0.1:  # Confidence threshold
                    x1, y1, x2, y2 = map(int, xyxy)

                    # Extract face
                    face_crop = image[y1:y2, x1:x2]

                    # Convert face crop to RGB for embedding extraction
                    face_crop_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)

                    # Get the embedding using DeepFace
                    try:
                        embeddings = DeepFace.represent(img_path=face_crop_rgb, model_name='Facenet', enforce_detection=False)
                        face_embeddings.append(embeddings[0]['embedding'])
                    except Exception as e:
                        print(f"Error processing detected face in {image_name}: {e}")

        # Save embeddings to a file with the original image name
        if face_embeddings:
            base_name = os.path.splitext(os.path.basename(image_path))[0]
            np.save(os.path.join('embeddings', f'{base_name}.npy'), face_embeddings)

        # Update progress bar
        pbar.update(1)

print("Processing complete.")

Processing Images: 100%|██████████| 1197/1197 [19:27<00:00,  1.03image/s] 

Processing complete.
